# 第5章 运输问题代码演示

## 发电厂问题

有3个发电厂向3个城市供电，其发电量分贝为25、40、30兆千瓦时。3个尝试的最大需求量为30、35和25兆千瓦时。这三个城市的每兆千瓦时店家如表所示。

| 电厂\\城市 |   1   |   2   |   3   |
| --------: | :---: | :---: | :---: |
|         1 |  600  |  700  |  400  |
|         2 |  320  |  300  |  350  |
|         3 |  500  |  480  |  450  |

在8月份期间，这3个城市的每一个都会增加20%的用点需求量，这部分电量可以从别的电网以每兆千瓦时1000的额外费用够得。这个电网不与第三个城市相连，电力公司希望确定增加电量的最经济的销售和采购计划。

In [1]:
from optlang import Model, Constraint, Variable, Objective

# 价格
price = [
    [600, 700, 400],
    [320, 300, 350],
    [500, 480, 450],
    [1000, 1000, None]
]

# 设定每条路线变量
edges = dict()
for i in range(1, 5):
    for j in range(1, 4):
        if i == 4 and j == 3:
            break
        else:
            edges[f'x{i}{j}'] = Variable(f'x{i}{j}', lb=0)

# 设定 优化函数
obj_func = 0
for i in range(1, 5):
    for j in range(1, 4):
        if i == 4 and j == 3:
            break
        else:
            obj_func += edges[f'x{i}{j}'] * price[i-1][j-1]
obj = Objective(obj_func, direction="min")

c_list = []

# 设定每个城市8月供电下限
for j, need in enumerate([30, 35, 25]):
    c_temp = 0
    for i in range(1, 5):
        if i == 4 and j == 2:
            break
        else:
            c_temp += edges[f'x{i}{j+1}'] 
    c_list.append(Constraint(c_temp, lb=need * 1.2))

# 设定每个城市非8月供电下限
for j, need in enumerate([30, 35, 25]):
    c_temp = 0
    for i in range(1, 4):
        if i == 4 and j == 2:
            break
        else:
            c_temp += edges[f'x{i}{j+1}'] 
    c_list.append(Constraint(c_temp, lb=need))

# 设定每个电厂供电上限
for i, product in enumerate([25, 40, 30]):
    c_temp = 0
    for j in range(1, 4):
        c_temp += edges[f'x{i+1}{j}']
    c_list.append(Constraint(c_temp, lb=0, ub=product))


model = Model(name='电厂模型')
model.objective = obj
model.add(c_list)

status = model.optimize()
print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.iteritems():
    print(var_name, "=", var.primal)

status: optimal
objective value: 49850.0
----------
x11 = 0.0
x12 = 0.0
x13 = 25.0
x21 = 30.0
x22 = 10.0
x23 = 0.0
x31 = 0.0
x32 = 25.0
x33 = 5.0
x41 = 6.0
x42 = 7.0
